# Proyek Klasifikasi Penumpang Titanic (KNN)

Notebook ini disusun untuk memenuhi tugas akhir mata kuliah Data Mining.
Tujuan proyek ini adalah memprediksi keselamatan penumpang kapal Titanic berdasarkan fitur seperti umur, kelas tiket, dan biaya, menggunakan algoritma **K-Nearest Neighbors (KNN)**.

## 1. Import Library
Langkah pertama adalah memanggil semua pustaka (library) Python yang dibutuhkan.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from IPython.display import display # Import modul untuk menampilkan tabel cantik

## 2. Upload Dataset (Khusus Google Colab)
Jalankan cell di bawah ini untuk mengunggah file `titanic_clean.csv`.

In [ ]:
from google.colab import files
import io

try:
    # Coba baca langsung jika file sudah ada
    df = pd.read_csv('titanic_clean.csv')
    print("File ditemukan di penyimpanan lokal.")
except FileNotFoundError:
    # Jika tidak ada, minta upload
    print("=== Silakan Upload File 'titanic_clean.csv' ===")
    uploaded = files.upload()
    filename = next(iter(uploaded))
    df = pd.read_csv(io.BytesIO(uploaded[filename]))

print("\n=== 5 Baris Pertama Data Titanic ===")
# Menggunakan display() agar tabel terlihat rapi
display(df.head())

## 3. Visualisasi Data
Melihat pola persebaran data antara Umur, Harga Tiket, dan Keselamatan.

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(x='Age', y='Fare', hue='Survived', data=df, palette='seismic')
plt.title('Sebaran Penumpang Titanic (Umur vs Harga Tiket)')
plt.xlabel('Umur')
plt.ylabel('Harga Tiket')
plt.legend(title='Status (1=Selamat)')
plt.show()

## 4. Splitting & Scaling Data (PENTING)
Agar akurasi tinggi, kita perlu:
1. Membagi data latih (80%) dan uji (20%).
2. Melakukan **Scaling** (menyamakan skala angka) agar KNN tidak bingung antara angka besar (Harga) dan angka kecil (Kelas).

In [ ]:
# 1. Pisahkan Fitur dan Target
X = df.drop('Survived', axis=1)
y = df['Survived']

# 2. Split Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Scaling (Standarisasi Data)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Data berhasil diproses dan di-scaling.")

## 5. Training & Evaluasi Model
Melatih model KNN dan melihat hasilnya.

In [ ]:
# Training
model = KNeighborsClassifier(n_neighbors=5)
model.fit(X_train_scaled, y_train)

# Prediksi
y_pred = model.predict(X_test_scaled)

# Akurasi
akurasi = accuracy_score(y_test, y_pred)
print(f"=== AKURASI MODEL: {akurasi * 100:.2f}% ===")

## 6. Tabel Hasil Prediksi (Visualisasi Tabel)
Di bawah ini adalah tabel perbandingan. 
* **Hijau**: Prediksi Benar
* **Merah**: Prediksi Salah

In [ ]:
# Siapkan Data
df_hasil = X_test.copy()
df_hasil['Status_Asli'] = y_test
df_hasil['Prediksi_Model'] = y_pred

# Ubah angka jadi teks
label_map = {0: 'Tidak Selamat', 1: 'Selamat'}
df_hasil['Status_Asli'] = df_hasil['Status_Asli'].map(label_map)
df_hasil['Prediksi_Model'] = df_hasil['Prediksi_Model'].map(label_map)

# Cek Benar/Salah
df_hasil['Hasil'] = np.where(df_hasil['Status_Asli'] == df_hasil['Prediksi_Model'], 'Benar', 'Salah')

# Fungsi untuk memberi warna
def warnai_tabel(row):
    if row['Hasil'] == 'Benar':
        return ['background-color: #d4edda; color: black'] * len(row)
    else:
        return ['background-color: #f8d7da; color: black'] * len(row)

print("Tabel 10 Data Pertama:")
cols = ['Age', 'Pclass', 'Fare', 'Status_Asli', 'Prediksi_Model', 'Hasil']

# MENAMPILKAN TABEL CANTIK (INTERAKTIF)
display(df_hasil[cols].head(10).style.apply(warnai_tabel, axis=1))

## 7. Simulasi Jack & Rose
Mari kita coba prediksi nasib Jack dan Rose menggunakan model yang sudah pintar ini.

In [ ]:
# Data Manual [Pclass, Sex, Age, SibSp, Parch, Fare]
jack = [3, 0, 20, 0, 0, 7.5]
rose = [1, 1, 17, 0, 1, 100.0]

# Masukkan ke DataFrame dan Scale
input_baru = pd.DataFrame([jack, rose], columns=X.columns)
input_scaled = scaler.transform(input_baru)

# Prediksi
hasil = model.predict(input_scaled)

print("=== HASIL SIMULASI ===")
for i, nama in enumerate(['Jack', 'Rose']):
    status = "SELAMAT" if hasil[i] == 1 else "TIDAK SELAMAT"
    print(f"Prediksi untuk {nama}: {status}")